In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [14]:
url="https://bridal.suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13107&sc=13109&sc=13110&sc=13111&sc=13112&sc=13115&cb=0.0&ct=9999999&et=10&ts=1&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1/?page={}"

In [28]:
# 変数flattened_dataに空のリストを作成する
flattened_data = []

for i in range(1, 136):
    print("Processing page:", i)
    target_url = url.format(i)
    print(target_url)
    r = requests.get(target_url)
    sleep(1)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    # 物件情報の取得
    cassette_items = soup.find_all("div", class_="cassetteitem")

    for cassette in cassette_items:
        property_name = cassette.find("div", class_="cassetteitem_content-title").text.strip()
        address = cassette.find("li", class_="cassetteitem_detail-col1").text.strip()
        access = ' / '.join([item.text.strip() for item in cassette.find("li", class_="cassetteitem_detail-col2").find_all("div")])
        age = cassette.find("li", class_="cassetteitem_detail-col3").text.strip()

        # 物件の各部屋情報を取得
        table_rows = cassette.find("table", class_="cassetteitem_other").find_all("tr")[1:]  # 最初の行はヘッダーなので除外

        for row in table_rows:
            columns = row.find_all("td")
            floor = columns[2].text.strip()
            rent = columns[3].text.strip()
            management_fee = columns[4].text.strip()
            layout = columns[6].text.strip()
            size = columns[7].text.strip()

            # フラット化されたデータをリストに追加
            flattened_data.append({
                "物件名": property_name,
                "住所": address,
                "アクセス": access,
                "築年数": age,
                "階数": floor,
                "家賃": rent,
                "管理費": management_fee,
                "間取り": layout,
                "広さ": size
            })

# DataFrameを作成
df = pd.DataFrame(flattened_data)

# DataFrameを表示
print(df)

Processing page: 1
https://bridal.suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13107&sc=13109&sc=13110&sc=13111&sc=13112&sc=13115&cb=0.0&ct=9999999&et=10&ts=1&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1/?page=1
Processing page: 2
https://bridal.suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13107&sc=13109&sc=13110&sc=13111&sc=13112&sc=13115&cb=0.0&ct=9999999&et=10&ts=1&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1/?page=2
Processing page: 3
https://bridal.suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13107&sc=13109&sc=13110&sc=13111&sc=13112&sc=13115&cb=0.0&ct=9999999&et=10&ts=1&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1/?page=3
Processing page: 4
https://bridal.suumo.jp/

In [29]:
df.shape

(9805, 9)

In [30]:
df.to_csv("suumodata.csv", index=None, encoding="utf-8-sig")